In [1]:
import pandas as pd

In [ ]:
# Sample GDP and population projections (you'll replace this with your actual data)
gdp_projections = {
    'Kenya': {2021: 90000, 2022: 95000, ..., 2050: 200000}  # Million USD
}
population_projections = {
    'Kenya': {2021: 55000000, 2022: 56000000, ..., 2050: 80000000}  
}

# Mode split data (replace with actual data if available)
mode_share_road_pkm = {'Kenya': {'BUS': 0.55, 'CAR': 0.2, 'MOTO': 0.1}}

In [2]:
country = input("Enter country name: ")  # Example: 'Kenya'

In [3]:
def get_base_pkm_per_mode(df, country):
    """
    Calculates base year pkm by mode based on data availability.

    Args:
        df (pandas.DataFrame): The input DataFrame.
        country (str): The selected country.

    Returns:
        dict: A dictionary containing base pkm per mode.
    """
    base_pkm_per_mode = {}
    modes = ['ROAD_PA_MOV', 'ROAD_PA_MOTORC', 'ROAD_PA_CAR', 'ROAD_PA_BUS']

    if all(mode in df['Data code'].unique() for mode in modes): 
        # Specific mode data is available
        for mode in modes:
            relevant_data = df[(df['Country name'] == country) & (df['Data code'] == mode)]
            base_year = relevant_data.columns[-1]  # Assume latest year
            base_pkm_per_mode[mode[7:]] = relevant_data[base_year].iloc[0]  
    else:
        # Use total and apply arbitrary mode split
        total_pkm_data = df[(df['Country name'] == country) & (df['Data code'] == 'ROAD_PA_MOV')]
        base_year = total_pkm_data.columns[-1]
        total_base_pkm = total_pkm_data[base_year].iloc[0]
        for mode, share in mode_share_road_pkm[country].items():
            base_pkm_per_mode[mode] = total_base_pkm * share

    return base_pkm_per_mode


def calculate_pkm_forecast(base_pkm_per_mode, gdp_projections, population_projections, country):
    """
    Forecasts passenger-km by mode for a given country.

    Args:
        base_pkm_per_mode (dict): Base year pkm data.
        gdp_projections (dict): GDP projections by country and year.
        population_projections (dict): Population projections by country and year.
        country (str): The selected country.

    Returns:
        dict: A nested dictionary containing pkm forecasts by mode and year.
    """
    pkm_forecast = {}  
    base_year = int(max(base_pkm_per_mode))  # Assuming base year is latest in the dictionary

    for year in range(base_year + 1, 2051):
        pkm_forecast[year] = {}
        gdp_growth = (gdp_projections[country][year] - gdp_projections[country][year - 1]) / gdp_projections[country][year - 1]
        pop_growth = (population_projections[country][year] - population_projections[country][year - 1]) / population_projections[country][year - 1]

        # TODO: Define how you will obtain dynamic elasticities by year
        gdp_elasticity = ... 
        pop_elasticity = ... 

        for mode, base_pkm in base_pkm_per_mode.items():
            pkm_forecast[year][mode] = base_pkm * (1 + gdp_elasticity * gdp_growth) * (1 + pop_elasticity * pop_growth)

    return pkm_forecast

'Kenya'